In [135]:
import pandas as pd
import numpy as np

# Importação e Cópia
Estou importando os dados diretamente do Gsheet, e na célula seguinte fazendo a cópia dos dataframes. A cópia serve para não ter o trabalho de importar dataframe da nuvem novamente.

In [136]:
DF_COMPRAS_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8/export?format=xlsx&id=117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8",
    sheet_name="Lançamento de NF Não Recorrente",
    usecols=[
        "Marca",
        "Unidade",
        "Valor Corrigido",
        "Mês Corrigido",
    ]
)

DF_DE_PARA_OFC = pd.read_excel(
    'https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=format=xlsx&id=1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ', 
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Marca 2024",
        "Unidade 2024"
    ]
)

c:\Users\fernando.luiz\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Cell E1954 is marked as a date but the serial value 6692727.0 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [137]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024.copy()
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC.copy()

# Correção do Mês
Aqui estou covertendo os meses faltantes pelo mês atual, e não só isso, convertendo os valores para inteiro.

In [138]:
from datetime import datetime

DF_COMPRAS_2024_COPY["Mês Corrigido"] = pd.to_numeric(DF_COMPRAS_2024_COPY["Mês Corrigido"], "coerce")
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].fillna(0)
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].astype("int")
DF_COMPRAS_2024_COPY.loc[DF_COMPRAS_2024_COPY["Mês Corrigido"] == 0, "Mês Corrigido"] = datetime.now().month

# Correção de Marca e Unidade
Agora, vamos atualizar a marca e unidade para os nomes de 2024, nesse processo estou fazendo:
- Retirandos os dados faltantes
- Aplicando função ```stip()``` na Marca e Unidade, para retirar os espaços em branco
- Criando uma coluna apenas com a Unidade
- Criando a Chave 2024
- Aplicando o DePara e puxando as Marcas e Unidades atualizadas

In [139]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Marca"].notna()]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Unidade"].notna()]

DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY[DF_DE_PARA_OFC_COPY["Chave 2024"].notna()]
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY.drop_duplicates()

DF_COMPRAS_2024_COPY["Marca"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Marca"].strip(), axis=1)
DF_COMPRAS_2024_COPY["Unidade"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Unidade"].strip(), axis=1)

DF_COMPRAS_2024_COPY["Unidade Atualizada"] = DF_COMPRAS_2024_COPY.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_COMPRAS_2024_COPY["Chave 2024"] = DF_COMPRAS_2024_COPY["Marca"] + DF_COMPRAS_2024_COPY["Unidade Atualizada"] 

DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "NUs")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Visita BE")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Salta")]

DF_COMPRAS_2024_DE_PARA = pd.merge(DF_COMPRAS_2024_COPY, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

In [140]:
DF_COMPRAS_2024_DE_PARA["Valor Corrigido"].sum()

10777449.23

# Dividindo o DataFrame
Nesse momento estou divindo meu DataFrame em dois, um com os erros, no qual vou tratar e unir novamente ao dataset original.

In [141]:
DF_ERROS = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].isna()]
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].notna()]

In [142]:
DF_ERROS["Marca"] = DF_ERROS.apply(lambda x: x["Unidade"].split(" ")[0], axis=1)

DF_ERROS["Unidade Atualizada"] = DF_ERROS.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_ERROS["Chave 2024"] = DF_ERROS["Marca"] + DF_ERROS["Unidade Atualizada"]
DF_ERROS.drop(columns=[
    "Marca 2024",
    "Unidade 2024",
], inplace=True)

DF_ERROS = pd.merge(DF_ERROS, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

# Unindo as Bases de Dados

In [143]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
]]

DF_COMPRAS_2024_DE_PARA["Ano"] = 2024
DF_COMPRAS_2024_DE_PARA["NO"] = "Ativos"
DF_COMPRAS_2024_DE_PARA["Categoria"] = "Supply"
DF_COMPRAS_2024_DE_PARA["Tipo Valor"] = "Capex"
DF_COMPRAS_2024_DE_PARA["Tipo"] = "Real"

DF_ERROS_ATUALIZADOS = DF_ERROS[DF_ERROS["Marca 2024"].notna()]
DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido"
]]

DF_ERROS_ATUALIZADOS["Ano"] = 2024
DF_ERROS_ATUALIZADOS["NO"] = "Ativos"
DF_ERROS_ATUALIZADOS["Categoria"] = "Supply"
DF_ERROS_ATUALIZADOS["Tipo Valor"] = "Capex"
DF_ERROS_ATUALIZADOS["Tipo"] = "Real"

DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
]]

DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
]]

In [144]:
DF_ATIVOS = pd.concat([DF_COMPRAS_2024_DE_PARA, DF_ERROS_ATUALIZADOS], axis=0)
DF_ATIVOS["Valor Corrigido"].sum() 

10767630.23

In [145]:
ATIVOS_ANTIGO = pd.read_excel("Ativos até Abril.xlsx")
DF_ATIVOS = DF_ATIVOS.rename(columns={
    "Mês Corrigido": "Mês",
    "Valor Corrigido": "Valor",
})

DF_ATIVOS = pd.concat([ATIVOS_ANTIGO, DF_ATIVOS], axis=0)
DF_ATIVOS.to_excel("Ativos 2024 (Padronizado).xlsx", index=False)